In [ ]:
 #%%capture
!pip install transformers>=4.41.2 accelerate>=0.31.0

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/Phi-3-mini-4k-instruct')

model = AutoModelForCausalLM.from_pretrained(
    'microsoft/Phi-3-mini-4k-instruct',
    device_map='cuda',
    torch_dtype='auto',
    trust_remote_code=False,
)

# Create a pipeline
generator = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=False,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


microsoft/Phi-3-mini-4k-instruct -- the model name or path on Hugging Face

device_map = cuda -- automatically places the model on your GPU (faster inference)

torch_dtype = auto -- automatically chooses the best PyTorch tensor type

trust_remote_code = False -- disables execution of custom model code for security reasons

- text-generation -- specifies the type of pipeline

- model=model -- uses the Phi-3-mini-4k-instruct model

- tokenizer=tokenizer -- uses the corresponding tokenizer to encode/decode text for the model.

- return_full_text=False - ensures that the output only contains the generated text, not the original prompt.

- max_new_tokens=500 - limits the maximum number of tokens the model can generate for a single response

- do_sample=False - disables random sampling (picks the token with highest probability)

In [2]:
prompt = 'Write an email apologizing to Samskruthi for the tragic production failure. Explain how it happened.'

output = generator(prompt)

print(output[0]['generated_text'])



Samskruthi,

I am writing to express my deepest apologies for the unfortunate production failure that occurred during our recent event. I understand the disappointment and frustration this may have caused, and I want


In [3]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLUActivation()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_featur



---



**Choosing a single token from the probability distribution (sampling / decoding)**

In [4]:
prompt = 'The capital of France is'

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors='pt').input_ids

# Tokenize the input prompt
input_ids = input_ids.to('cuda')

# Get the output of the model before the lm_head
model_output = model.model(input_ids)

# Get the output of the lm_head
lm_head_output = model.lm_head(model_output[0])

In [5]:
token_id = lm_head_output[0,-1].argmax(-1)
tokenizer.decode(token_id)

'Paris'

In [6]:
model_output[0].shape

torch.Size([1, 5, 3072])

In [7]:
lm_head_output.shape

torch.Size([1, 5, 32064])

# **Speeding up generation by caching keys and values**

In [9]:
prompt1 = 'Write a very long email apologizing to Samskruthi for the tragic production failure. Explain how it happened.'

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors='pt').input_ids
input_ids = input_ids.to('cuda')

In [10]:
output = generator(prompt1)

print(output[0]['generated_text'])



Samskruthi,

I am writing to you with a heavy heart to express my deepest apologies for the tragic production failure that occurred during our recent project. I understand that this has caused a great deal of disappoint


In [11]:
%%timeit -n 1
# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=True
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


4.49 s ± 252 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


- %%timeit -n 1
measures execution time of the cell.
-n 1 means: run once (by default, it runs multiple times and averages).
Output shows something like:
1 loop, best of 1: 1.23 s per loop meaning your model took ~1.23 seconds to generate text.

- model.generate() calls your language models text generation method

  - input_ids=input_ids Tokenized version of your input text (converted from words to token IDs) the model uses this as the prompt to start generation.

  - max_new_tokens=100 model will generate up to 100 new tokens after the prompt.

  - use_cache=True Enables the key-value cache in transformer layers for faster generation.


- During generation, each new token reuses attention results from previous steps.
Without cache, the model would recompute everything at every step

In [12]:
%%timeit -n 1
# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=False
)

30.9 s ± 220 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
